In [1]:
# Only run this once
import os
os.chdir('..')

In [2]:
import numpy as np
import scipy
from scipy import signal
import mne
import glob
from sklearn.decomposition import PCA
import csv
import os
from sklearn.model_selection import GridSearchCV

## Bandpower

In [8]:
# Calculate the band power over a time series

def bandpower(x, fs, fmin, fmax):
    """
    Returns the band power over the specified frequency interval
    
    x - input time series (1D array)
    fs - sampling frequency
    fmin - min frequency
    fmax - max frequency
    """
    
    f, Pxx = signal.periodogram(x, fs=fs)
    ind_min = scipy.argmax(f > fmin) - 1
    ind_max = scipy.argmax(f > fmax) - 1
    return scipy.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

In [9]:
def get_datasets(patient_type_location, recording_type_expression):
    """
    Returns relevant datasets (f.e. all right-hand recordings of patients with pain) as a list of np arrays
    First parameter should be a regex for location, second parameter should be a regex for dataset type
    E.g. if right-hand movement datasets of patients with pain are in /data/pp/ and their file names contain '_RH_'
    then patient_type_location=/data/pp/ and recording_type_expression='_RH_'
    """
    
    # Find locations of matching dataset files
    if recording_type_expression != l_new:
        sets_locations = glob.glob(patient_type_location + recording_type_expression + suffix)
    else:
        # For the newer (PDP/PP) dataset we had to use a separate expression with includes the file extension
        sets_locations = glob.glob(patient_type_location + recording_type_expression)
    
    sets = []
    for path in sets_locations: 
        sets.append(mne.io.read_epochs_eeglab(path))
        
    return np.array(np.array([(patient._data) for patient in sets]))

In [10]:
# Calculate bandpower for all channels for a patient

bands = [(4, 8), (8, 13), (13, 30)]
time_series_index = range(1250)[375:500]

def channels_bandpower(channels, bands, fs=250):
    b = bandpower
    return np.array(list(map( lambda arr: [b(arr[time_series_index], fs, band[0], band[1]) for band in bands], channels)))

## Define Locations

**Dataset Locations & Expressions**

In [12]:
root = root = './../../'
suffix = '*.set'

# Old (PP/PNP datasets)
location_healthy = root + 'data/raw/HV/*/'
location_pain = root + 'data/raw/PP/*/'
location_nopain = root + 'data/raw/PnP/*/'

# New (PDP/PNP datasets)
location_pwp = root + 'data_new/raw/PwP/*/'
location_pdp = root + 'data_new/raw/PdP/*/'
location_pnp = root + 'data_new/raw/PnP/*/'

rh = '*_RH*'
lh = '*_LH*'
l_new = '*_L.set'   # NO SUFFIX
l_old = '*_L_*'


# As an example, get paths of all PP/PNP datasets from left-hand movements
sets_healthy_lh = glob.glob(location_pain + lh + suffix)
sets_healthy_lh

['./../../data/raw/PP\\PP1\\PP1_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP10\\PP10_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP11\\PP11_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP2\\PP2_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP3\\PP3_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP4\\PP4_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP5\\PP5_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP6\\PP6_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP7\\PP7_F1_LH_Removed_ICA.set',
 './../../data/raw/PP\\PP9\\PP9_F1_LH_Removed_ICA.set']

In [13]:
# Read chosen datasets
pp_lh_raw = get_datasets(location_pain, lh)
pnp_lh_raw = get_datasets(location_nopain, lh)

Extracting parameters from ./../../data/raw/PP\PP1\PP1_F1_LH_Removed_ICA.set...


<ipython-input-9-916861aa92ff>:18: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


56 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../data/raw/PP\PP10\PP10_F1_LH_Removed_ICA.set...


<ipython-input-9-916861aa92ff>:18: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


60 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../data/raw/PP\PP11\PP11_F1_LH_Removed_ICA.set...


...

59 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../data/raw/PnP\PnP8\PnP8_F1_LH_Removed_ICA.set...


<ipython-input-9-916861aa92ff>:18: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


58 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../data/raw/PnP\PnP9\PnP9_F1_LH_Removed_ICA.set...
50 matching events found


<ipython-input-9-916861aa92ff>:18: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


No baseline correction applied
Not setting metadata
0 projection items activated
Ready.


In [14]:
# Test the entry for a patient should have shape (n_repetitions, n_channels, n_readings)
pp_lh_raw[8].shape

(53, 61, 1250)

## Apply bandpower

In [15]:
# Apply bandpower to chosen datasets
pp_lh_bp = np.array([np.array([channels_bandpower(repetition, bands) for repetition in patient]) for patient in pp_lh_raw])
pnp_lh_bp = np.array([np.array([channels_bandpower(repetition, bands) for repetition in patient]) for patient in pnp_lh_raw])## SVM

In [16]:
# Get the total number of repetitions for each class
pp_count = np.vstack(pp_lh_bp).shape[0]
pnp_count = np.vstack(pnp_lh_bp).shape[0]

# print(pp_count, pnp_count)

**Concatenate the two classes: patients with/without pain**

In [17]:
pp_and_pnp_bp = np.concatenate((pp_lh_bp, pnp_lh_bp))
pp_and_pnp_bp.shape

(19,)

**Set Aside for Training/Testing**

In [18]:
def test_setup(test_index, total_size):
    """
    Returns a pair consisting of boolean (True is test patient is PP) and test label
    Labels are 1 for pain, 0 for no pain
    """
    
    test_is_pp = test_index < len(pp_lh_bp)
    test_label = 1 if test_is_pp else 0
    return test_is_pp, test_label


def get_train_test(data, test_index):
    """
    Splits into train and test sets based on the index of the test patient
    Returns pair of test and train
    """
    
    return data[test_index], np.delete(data, test_index)


def get_pp_pnp_length(pp_count, pnp_count, test_count, test_is_pp):
    """ Returns pair of the lengths of PP train data and respectively PNP train data """
    
    pp_train_len = pp_count if not test_is_pp else pp_count - test_count
    pnp_train_len = pnp_count if test_is_pp else pnp_count - test_count
    return pp_train_len, pnp_train_len


def ravel_all_trials(data, channels):
    """
    Ravel first dimension so that trials from all patients are treated separately; select channels
    """
    return np.array(list(map(np.ravel, data[:, channels, :])))

In [19]:
test_index = 4
test_is_pp, test_label = test_setup(test_index, len(pp_lh_bp))
test_label

1

In [20]:
test_p, train_p = get_train_test(pp_and_pnp_bp, test_index)
test_p.shape

(51, 61, 3)

In [21]:
train_p_separated = np.vstack(train_p)
train_p_separated.shape

(954, 61, 3)

In [22]:
pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
pp_train_len

474

Define a multiplier for all features If 1 then data is unaltered?
Why??

In [23]:
mul = 1

In [24]:
selected_channels = [10, 11]
#what have these channels been selected for? and how?

In [25]:
train = ravel_all_trials(train_p_separated, selected_channels)
train.shape


(954, 6)

In [26]:
test = ravel_all_trials(test_p, selected_channels)
test.shape

(51, 6)

## Classification SVM

In [27]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [28]:
import warnings
warnings.filterwarnings('ignore')

**PCA**

In [29]:
def pca(array, n_components=5):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(array)

Define number of PCA components to use

note: we want to try to incorporate a loop in which we test which number of components will provide best acuracy, is 3 best?

In [30]:
n_components = 3

In [31]:
pca_components = 3

In [32]:
pca(train, n_components).shape

(954, 3)

In [33]:
def classify_nusvm_with_xvalid(data_pp_bp, data_pnp_bp, nu, selected_channels, test_index, mul, verbose=True):
    """
    Leaves out patient at test_index and trains a linear SVM classifier on all other patients
    Validates classifier on patient at test_index
    Returns accuracy over all repetitions for the test patient
    """
    
    data_bp = np.concatenate((data_pp_bp, data_pnp_bp))
    
    test_is_pp, test_label = test_setup(test_index, len(data_pp_bp))
    test_p, train_p = get_train_test(data_bp, test_index)
    train_p_separated = np.vstack(train_p)
    pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
    
    train = pca(ravel_all_trials(train_p_separated, selected_channels) * mul, n_components=pca_components)
    test = pca(ravel_all_trials(test_p, selected_channels) * mul, n_components=pca_components)
    
    labels = [1] * pp_train_len + [0] * pnp_train_len
    test_labels = [test_label] * len(test)
    
    if verbose:
        print('Test index', test_index, 'Preparing to classify set of', pp_train_len, 'PP and', pnp_train_len, 'PNP.')
    
    clas = svm.NuSVC(nu=nu, kernel='linear')
    clas.fit(train, labels)
    train_acc = clas.score(train, labels)
    test_acc = clas.score(test, test_labels)
    
    if verbose:
        print('Train score:', train_acc, '  Test score:', test_acc)
    
    return test_acc

## Logging

In [34]:
def log_result(file, name, accuracy, patients_correct, patients_total, set_name, channels, nu, notes):
    """
    Log the result of an experiment in a csv file with the following information:
        experiment name, accuracy (over all repetitions), number of correct patients/total number of patients, 
        dataset name, selected channels, notes
    """
    writer = csv.writer(file)
    writer.writerow([name, str(accuracy*100) + '%', ' ' + str(patients_correct) + '/' + str(patients_total), set_name, channels, nu, notes])
    

**Automated channel selection attempt** (single nu value - 0.8)

manually set nu value here - see cell below for automated nu selection

In [64]:
# Log results in a csv file
file = open('all_results/auto_bandpower_pca_svm_results.csv', 'a', newline='')
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8, 8-13, 13-30 over 375-500'
notes_c = ', n_components='

previous_channels = []
nu = 0.80
prev_max_acc = 0

for i in range(61):
    max_acc = {'index': 0, 'value': 0}
    for channel in range(61):
        if channel in previous_channels:
            continue
        total_score = 0
        correct_patients = 0
        for i in range(len(pp_and_pnp_bp)):
            score = classify_nusvm_with_xvalid(pp_lh_bp, pnp_lh_bp, nu, previous_channels + [channel], i, mul, verbose=False)
            total_score += score
            if score > 0.5:
                correct_patients += 1

        avg_score = total_score/len(pp_and_pnp_bp)
        print(channel, nu, avg_score, correct_patients)

        # log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'LH', str(previous_channels + [channel]), str(nu), notes + notes_c + str(n_components))

        if avg_score > max_acc['value']:
            max_acc['index'] = channel
            max_acc['value'] = avg_score

    if max_acc['value'] < prev_max_acc:
        break
    
    prev_max_acc = max_acc['value']
    previous_channels = previous_channels + [max_acc['index']]
    print(previous_channels)
    
file.close()
        
print('Max accuracy:', previous_channels, prev_max_acc)

0 0.8 0.43221973990348295 6
1 0.8 0.5083079377704592 8
2 0.8 0.5123533784065923 6
3 0.8 0.6443586535477203 12
4 0.8 0.6054515614965069 10
5 0.8 0.6176668111293852 11
6 0.8 0.7114607072563293 14
7 0.8 0.4386997466919973 7
8 0.8 0.7283249311537496 14
9 0.8 0.6500500830600486 13
10 0.8 0.7472253332559086 16
11 0.8 0.7580445075416968 16
12 0.8 0.8136648532670658 17
13 0.8 0.5908856130615876 11
14 0.8 0.7447968965045755 15
15 0.8 0.7168688370409613 14
16 0.8 0.6430818991856381 11
17 0.8 0.7305580352457283 14
18 0.8 0.5676344063669768 11
19 0.8 0.517204624451153 9
20 0.8 0.5103671606268085 7
21 0.8 0.6116770428696131 12
22 0.8 0.6319584582103235 12
23 0.8 0.6102019200261136 11
24 0.8 0.697890476717918 14
25 0.8 0.4976969401851411 10
26 0.8 0.6661626350442282 12
27 0.8 0.7565921497377482 14
28 0.8 0.7742208667269285 15
29 0.8 0.8145315739010375 17
30 0.8 0.8202691614521557 17
31 0.8 0.7655322784951691 16
32 0.8 0.7868422033556871 16
33 0.8 0.5912589109406111 12
34 0.8 0.7485936741703056 14
35

**Automated channel selection attempt** (with multiple nu values)

automates channel selection and checks for nu value (in given range) which gives greatest accuracy 

In [36]:
# Log results in a csv file
file = open('auto_bandpower_pca_svm_results.csv', 'a', newline='')
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8, 8-13, 13-30 over 375-500'
notes_c = ', n_components='

max_acc_overall = {'channels': [], 'value': 0, 'nu': 0}

for param_nu in np.arange(0.6, 0.89, 0.01):
    previous_channels = []
    prev_max_acc = 0
    for i in range(61):
        max_acc = {'index': 0, 'value': 0}
        for channel in range(61):
            if channel in previous_channels:
                continue
            total_score = 0
            correct_patients = 0
            for i in range(len(pp_and_pnp_bp)):
                score = classify_nusvm_with_xvalid(pp_lh_bp, pnp_lh_bp, param_nu, previous_channels + [channel], i, mul, verbose=False)
                total_score += score
                if score > 0.5:
                    correct_patients += 1

            avg_score = total_score/len(pp_and_pnp_bp)
            print(channel, param_nu, avg_score, correct_patients)

            log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'LH', str(previous_channels + [channel]), str(param_nu), notes + notes_c + str(n_components))

            if avg_score > max_acc['value']:
                max_acc['index'] = channel
                max_acc['value'] = avg_score
                max_acc['nu'] = param_nu

        if max_acc['value'] < prev_max_acc:
            break
        
        prev_max_acc = max_acc['value']
        previous_channels = previous_channels + [max_acc['index']]
        print(previous_channels, '{:.2f}'.format(prev_max_acc))
    
    if prev_max_acc > max_acc_overall['value']:
        max_acc_overall['channels'] = list(previous_channels)
        max_acc_overall['value'] = prev_max_acc
        max_acc_overall['nu'] = param_nu
        
    print('Max Accuracy:', max_acc_overall)
    
print('Max Accuracy:', max_acc_overall)
    
file.close()

0 0.6 0.49290619032530253 9
1 0.6 0.6207415137105428 13
2 0.6 0.5569173809991566 10
3 0.6 0.6852622844561772 13
4 0.6 0.5895462813092444 10
5 0.6 0.46054022736788025 8
6 0.6 0.4553749216870335 10
7 0.6 0.47066750610649205 9
8 0.6 0.415378926272502 9
9 0.6 0.6291795856302228 12
10 0.6 0.3383334078566289 5
11 0.6 0.5546826933342706 10
12 0.6 0.45816076824278196 7
13 0.6 0.6478685651859656 12
14 0.6 0.49245868987036856 8
15 0.6 0.4331724947335334 8
16 0.6 0.4009964705163945 8
17 0.6 0.37570655916855783 8
18 0.6 0.556294941141344 10
19 0.6 0.5627307848753017 11
20 0.6 0.4964054442805045 9
21 0.6 0.6488667314065198 14
22 0.6 0.47070140715103237 10
23 0.6 0.5476123935968179 10
24 0.6 0.4915342117147745 11
25 0.6 0.5211286526580483 9
26 0.6 0.36123587284470005 7
27 0.6 0.3841731844923054 8
28 0.6 0.34810765677797134 6
29 0.6 0.5045999471071825 10
30 0.6 0.3839100635549122 6
31 0.6 0.5442275963258386 10
32 0.6 0.4582178158445663 10
33 0.6 0.4631517773052879 9
34 0.6 0.40417615452629135 7
35 0.

45 0.61 0.30908260222693323 6
46 0.61 0.34711028842137237 7
47 0.61 0.458315323166341 9
48 0.61 0.4244704738404736 9
49 0.61 0.5419053475150579 10
50 0.61 0.4196453129969545 8
51 0.61 0.5722960517361253 11
52 0.61 0.2935500515995872 6
53 0.61 0.2715348208248817 5
54 0.61 0.49418587838230144 9
55 0.61 0.4229186075915827 7
56 0.61 0.42354737198698894 8
57 0.61 0.309304356414289 4
58 0.61 0.4943348924598232 9
59 0.61 0.3040291331686865 5
60 0.61 0.44275861477495926 9
[19] 0.68
0 0.61 0.6680563441076056 14
1 0.61 0.6315032195897 13
2 0.61 0.6679164580156404 13
3 0.61 0.634553706957805 12
4 0.61 0.6525176927055552 13
5 0.61 0.6885744492827456 13
6 0.61 0.5232189433618434 10
7 0.61 0.7136150761484273 14
8 0.61 0.42171841957143574 8
9 0.61 0.5324956307460748 11
10 0.61 0.31126010135710575 5
11 0.61 0.5669107538207349 12
12 0.61 0.3905124267429175 7
13 0.61 0.6946965395430977 14
14 0.61 0.5773374474816712 11
15 0.61 0.4407068568575503 8
16 0.61 0.6457058916496424 13
17 0.61 0.3575596980441446 

22 0.62 0.4656504450292437 9
23 0.62 0.5184877910807177 11
24 0.62 0.4377480999735117 9
25 0.62 0.6150442669892385 12
26 0.62 0.44095682815996007 8
27 0.62 0.49751150868153404 8
28 0.62 0.44777903786930173 8
29 0.62 0.5256349753011943 9
30 0.62 0.48556878877455223 9
31 0.62 0.5702626142751882 10
32 0.62 0.47618477758807876 8
33 0.62 0.5095288437782508 9
34 0.62 0.4678163537986726 9
35 0.62 0.45898044820324574 9
36 0.62 0.5266770890659092 10
37 0.62 0.48015036203160516 8
38 0.62 0.5173292468538175 10
39 0.62 0.5226829624533472 9
40 0.62 0.4607457385151384 9
41 0.62 0.47009071125564733 9
42 0.62 0.44894083502949544 9
43 0.62 0.5004126514941724 9
44 0.62 0.4147790539558138 7
45 0.62 0.5146725814599359 9
46 0.62 0.4798067371394537 9
47 0.62 0.528434204167751 10
48 0.62 0.5291941716001095 9
49 0.62 0.5225568084267141 9
50 0.62 0.5288753450184901 10
51 0.62 0.5283477818033605 9
52 0.62 0.48547335831839666 9
53 0.62 0.4284958017976509 8
54 0.62 0.5272621827627235 10
55 0.62 0.413676857686372 

59 0.63 0.524685889834047 11
60 0.63 0.6791371399108724 14
{'channels': [3, 11, 19], 'value': 0.7575489032163707, 'nu': 0.6}
0 0.64 0.5312808504924651 12
1 0.64 0.5497373288844657 13
2 0.64 0.6738240212672832 14
3 0.64 0.5560523608681603 9
4 0.64 0.4917247187509933 9
5 0.64 0.4434535788774193 9
6 0.64 0.4410200702497775 7
7 0.64 0.40968897232556695 8
8 0.64 0.3296371364191326 8
9 0.64 0.5147313000545409 9
10 0.64 0.42774467886478473 7
11 0.64 0.6378256987761841 12
12 0.64 0.4243016687925872 7
13 0.64 0.4142830570789964 8
14 0.64 0.46847894581862914 9
15 0.64 0.43061806302920363 7
16 0.64 0.41399256609217655 8
17 0.64 0.3663190986924889 6
18 0.64 0.5513918167004287 11
19 0.64 0.5177492751069964 10
20 0.64 0.4877602688850908 8
21 0.64 0.539352611820796 11
22 0.64 0.45276525492510017 9
23 0.64 0.48639077131607694 9
24 0.64 0.4921837300406405 11
25 0.64 0.4754491320131031 9
26 0.64 0.37676171373460127 8
27 0.64 0.47488095038360084 9
28 0.64 0.4048684959720414 9
29 0.64 0.5286482121581234 1

30 0.65 0.45303332825642767 10
31 0.65 0.5397886875683334 10
32 0.65 0.4543728304787858 10
33 0.65 0.44042979515472586 8
34 0.65 0.4407699163780588 11
35 0.65 0.47624628048019635 9
36 0.65 0.5008619231939889 9
37 0.65 0.5441172143578827 10
38 0.65 0.4966043008361065 10
39 0.65 0.5076650632746511 10
40 0.65 0.49508426347696005 11
41 0.65 0.5023699589639923 10
42 0.65 0.47268909475437515 10
43 0.65 0.4305995780354307 9
44 0.65 0.45532305261621747 9
45 0.65 0.4277177775778152 9
46 0.65 0.4238529944213126 10
47 0.65 0.4304055489640874 10
48 0.65 0.4954998292919321 11
49 0.65 0.48328383372274475 9
50 0.65 0.4880501124177307 11
51 0.65 0.4752161732082064 11
52 0.65 0.45096681331812505 8
53 0.65 0.43298028352615286 8
54 0.65 0.4611355634510663 8
55 0.65 0.29974346139998564 4
56 0.65 0.5106329429121326 10
57 0.65 0.47217107265080027 8
58 0.65 0.36669157964240867 5
59 0.65 0.4611823256669171 8
60 0.65 0.4890568998532469 10
{'channels': [3, 11, 19], 'value': 0.7575489032163707, 'nu': 0.6}
0 0.66

...

14 0.8700000000000002 0.8232284077678926 16
15 0.8700000000000002 0.830355981572013 17
16 0.8700000000000002 0.6781628735773229 12
17 0.8700000000000002 0.822232883148279 15
18 0.8700000000000002 0.810444885340513 16
19 0.8700000000000002 0.8264533561602473 16
20 0.8700000000000002 0.805160138384743 15
21 0.8700000000000002 0.8003467119136859 15
22 0.8700000000000002 0.7921940675095622 15
23 0.8700000000000002 0.7339472145361904 14
24 0.8700000000000002 0.7772940396515082 15
25 0.8700000000000002 0.6548458554679447 12
26 0.8700000000000002 0.7918702621308304 15
27 0.8700000000000002 0.7689354877876663 15
28 0.8700000000000002 0.7769828709510095 15
29 0.8700000000000002 0.7956290416706306 16
30 0.8700000000000002 0.8293726454753451 16
31 0.8700000000000002 0.8515774267043645 17
32 0.8700000000000002 0.8179314177069216 16
33 0.8700000000000002 0.8423166292938197 17
34 0.8700000000000002 0.724045588999238 13
35 0.8700000000000002 0.8161410497195029 17
36 0.8700000000000002 0.7062846752918

25 0.8700000000000002 0.8715428544137112 16
26 0.8700000000000002 0.8620784550964972 16
27 0.8700000000000002 0.8758506306333785 16
28 0.8700000000000002 0.8768991109673403 16
29 0.8700000000000002 0.876051403463286 16
30 0.8700000000000002 0.8749437581891101 16
31 0.8700000000000002 0.8752366731390233 16
32 0.8700000000000002 0.8747889594275002 16
34 0.8700000000000002 0.8739855879096102 16
35 0.8700000000000002 0.8742046813949574 16
36 0.8700000000000002 0.8762339418526032 16
37 0.8700000000000002 0.8241589013100703 15
38 0.8700000000000002 0.8748834561047649 16
39 0.8700000000000002 0.8735765898679656 16
40 0.8700000000000002 0.8703892350712373 16
41 0.8700000000000002 0.8669710801504669 16
42 0.8700000000000002 0.8525819459506031 16
43 0.8700000000000002 0.8734982071919418 16
44 0.8700000000000002 0.8631188579909734 16
45 0.8700000000000002 0.8768826223774445 16
46 0.8700000000000002 0.8740824852244842 16
48 0.8700000000000002 0.8738655983933359 16
49 0.8700000000000002 0.874204681

36 0.8800000000000002 0.7571155188180678 14
37 0.8800000000000002 0.8072736234591795 16
38 0.8800000000000002 0.8687077168534618 19
39 0.8800000000000002 0.8110679947537461 17
40 0.8800000000000002 0.773958873416348 15
41 0.8800000000000002 0.6985145254696478 13
42 0.8800000000000002 0.70701106422992 14
43 0.8800000000000002 0.830435003944646 17
44 0.8800000000000002 0.7075169731015464 13
45 0.8800000000000002 0.8686516572436782 18
46 0.8800000000000002 0.8670023367213866 17
47 0.8800000000000002 0.9023824574106316 18
48 0.8800000000000002 0.8920488267305919 19
49 0.8800000000000002 0.8616489308672506 18
50 0.8800000000000002 0.8677450939977194 17
51 0.8800000000000002 0.8591693039008558 17
52 0.8800000000000002 0.8916319883652006 17
53 0.8800000000000002 0.8929225749277105 17
54 0.8800000000000002 0.8768796322221561 16
55 0.8800000000000002 0.8990220300344589 18
56 0.8800000000000002 0.8044770001048357 17
57 0.8800000000000002 0.9038905101334515 18
58 0.8800000000000002 0.904799766458

44 0.8900000000000002 0.46101183795224676 9
45 0.8900000000000002 0.5009718844231495 11
46 0.8900000000000002 0.5443128877254363 10
47 0.8900000000000002 0.41427611813267673 6
48 0.8900000000000002 0.5266105256091332 12
49 0.8900000000000002 0.46973911930786727 7
50 0.8900000000000002 0.5456313533789494 10
51 0.8900000000000002 0.45981354455091833 7
52 0.8900000000000002 0.6969655308871089 13
53 0.8900000000000002 0.6132340326014445 12
54 0.8900000000000002 0.7490410741488858 15
55 0.8900000000000002 0.5064355952916986 8
56 0.8900000000000002 0.581377157048872 10
57 0.8900000000000002 0.5986229642639286 11
58 0.8900000000000002 0.4881383701305798 9
59 0.8900000000000002 0.5914362894568829 12
60 0.8900000000000002 0.5374900865810351 10
[54] 0.75
0 0.8900000000000002 0.7415941135188842 14
1 0.8900000000000002 0.7437143799203344 14
2 0.8900000000000002 0.747077475312222 15
3 0.8900000000000002 0.7248434060391306 14
4 0.8900000000000002 0.6771872647930002 14
5 0.8900000000000002 0.70620990

52 0.8900000000000002 0.679505926928991 13
55 0.8900000000000002 0.6846075020024296 14
56 0.8900000000000002 0.7416232797860959 15
57 0.8900000000000002 0.7527359355463082 15
58 0.8900000000000002 0.6928426678944198 14
59 0.8900000000000002 0.7495479961342816 15
60 0.8900000000000002 0.7575702570793468 15
{'channels': [46, 32, 52], 'value': 0.9618135168651257, 'nu': 0.8300000000000002}
Max Accuracy: {'channels': [46, 32, 52], 'value': 0.9618135168651257, 'nu': 0.8300000000000002}
